## Notebook to get parameter info from LOOKUP TABLES for MAP (Landuse and Irrigation)
### Script combines all parameters together and makes a .csv file with upper and lower bounds to use with pyemu

In [1]:
# Most recent run:
# 1/29/2020
# 8/10/2020
# 9/3/2020 (LU), 9/9/2020 (IRR)

In [2]:
import os
import numpy as np
import time
import pandas as pd
import shutil

### Set path and version for input Excel sheet:

In [3]:
# Location of lookup table spreadsheets:

pardir = r'E:\UMID_Data\0188_offline\modeldev\PARS\TABLES'

#landuse_version = 'YETI_Landuse_lookup_table_V4'
#irrigation_version = 'YETI_Irrigation_lookup_table_V2'
landuse_version = 'YETI_LU_lookup_table_MOREPARS'
irrigation_version = 'YETI_IRR_lookup_table_MOREPARS'

lufile = landuse_version + '.xlsx'
irrfile = irrigation_version + '.xlsx'

In [4]:
### Goal is to a) read excel file and find parameters, b) make a dataframe with PARUBND, PARLBND, etc., 
### c) get data from a second excel file (irrigation), and do the same.

### Starting with LU Table: Uses the BLUE colored tabs for Parnme and ParVal1

In [46]:
# Find starting values for each of these - read in two long lists and delete "--"
infile = os.path.join(pardir,lufile)

pnme = pd.read_excel(infile, 'Parnme', skiprows=3, na_values='--')
pval = pd.read_excel(infile, 'ParVal1', skiprows=3, na_values='--')
del pnme['Unnamed: 0']
del pval['Unnamed: 0']
#print (pval.info())
#print (pnme.info())

In [47]:
# Get the ParName list
test = pnme[~pnme.applymap(np.isreal).all(1)]
# This reduces the columns to ones that have text values:
test2 = test.select_dtypes(include='object')
print (test2.info())
test5 = pd.melt(test2, id_vars='description', value_name='ParName').dropna()
test6 = test5.apply(lambda x: x.str.strip('~'))
test7 = test6.apply(lambda x: x.str.strip()).reset_index()

# Create index location for the values:
test7['Index_Loc'] = test7['description'] + ":" + test7['variable']
test8 = test7.sort_values('Index_Loc')
print (test8.tail())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127 entries, 0 to 126
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   description      127 non-null    object
 1   cn_1             32 non-null     object
 2   cn_2             32 non-null     object
 3   cn_3             32 non-null     object
 4   cn_4             32 non-null     object
 5   cn_5             32 non-null     object
 6   cn_6             32 non-null     object
 7   cn_7             32 non-null     object
 8   max_net_infil_1  32 non-null     object
 9   max_net_infil_2  32 non-null     object
 10  max_net_infil_3  32 non-null     object
 11  max_net_infil_4  32 non-null     object
 12  max_net_infil_5  32 non-null     object
 13  max_net_infil_6  32 non-null     object
 14  max_net_infil_7  32 non-null     object
 15  rz_1             32 non-null     object
 16  rz_2             32 non-null     object
 17  rz_3             32 non-null     ob

In [48]:
# Get the ParVal list
t3 = pval[~pval.applymap(np.isreal).all(1)]
t4 = t3.select_dtypes(include=['float64','object'])
#print (test2.info())
t5 = pd.melt(t4, id_vars='description', value_name='ParVal').dropna()
t5 = t5.applymap(lambda x: str(x).strip() if len(str(x).strip()) else None)  # Important - stripped off leading spaces

# Create index location for the values:
t5['Index_Loc'] = t5['description'] + ":" + t5['variable']
t6 = t5[['Index_Loc','ParVal']].sort_values('Index_Loc')
t6.ParVal = t6.ParVal.astype(np.float64)
print (t6.head())

       Index_Loc   ParVal
69   Barren:cn_1  81.4000
196  Barren:cn_2  88.4308
323  Barren:cn_3  92.3554
450  Barren:cn_4  93.8992
577  Barren:cn_5  81.4000


In [49]:
# Join these together, remove the many duplicated items:
parvals = pd.merge(test8, t6, how='inner', on='Index_Loc', 
                   right_index=False, left_index=False, copy=False).drop_duplicates(['ParName','ParVal'])
print (parvals.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277 entries, 0 to 671
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        277 non-null    int64  
 1   description  277 non-null    object 
 2   variable     277 non-null    object 
 3   ParName      277 non-null    object 
 4   Index_Loc    277 non-null    object 
 5   ParVal       277 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 15.1+ KB
None


In [50]:
# Look for duplicated ParName with different ParVals - fix in spreadsheet:

# ADD CODE HERE to find items, fix in spreadsheet until they are gone.

In [51]:
mask = parvals.ParName.duplicated(keep=False)
dupes = (parvals[mask]).sort_values(['ParName'])
mask2 = dupes.duplicated(['ParName','ParVal'], keep=False)
# Want this to be empty:
realdupes = (dupes[~mask2])
realdupes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        0 non-null      int64  
 1   description  0 non-null      object 
 2   variable     0 non-null      object 
 3   ParName      0 non-null      object 
 4   Index_Loc    0 non-null      object 
 5   ParVal       0 non-null      float64
dtypes: float64(1), int64(1), object(4)
memory usage: 0.0+ bytes


In [52]:
realdupes.to_csv('landuse_pars_to_clean_up.csv')

### REPEAT for the Irrigation table:

In [39]:
# Find starting values for each of these - read in two long lists and delete "--"
infile = os.path.join(pardir,irrfile)

pnme2 = pd.read_excel(infile, 'Parnme', skiprows=2, na_values='--')
pval2 = pd.read_excel(infile, 'ParVal1', skiprows=2, na_values='--')
del pnme2['Unnamed: 0']
del pval2['Unnamed: 0']
#print (pval.info())

In [40]:
# Get the ParName list
irr1 = pnme2[~pnme2.applymap(np.isreal).all(1)]
irr2 = irr1.select_dtypes(include='object')
#print (test2)
irr3 = pd.melt(irr2, id_vars='description', value_name='ParName').dropna()
irr4 = irr3.apply(lambda x: x.str.strip('~'))
irr5 = irr4.apply(lambda x: x.str.strip()).reset_index()

# Create index location for the values:
irr5['Index_Loc'] = irr5['description'] + ":" + irr5['variable']
print (irr5.tail())

     index          description variable       ParName  \
449   1977         Mixed Forest    tew_7  decid_cd_tew   
450   1979            Shrubland    tew_7  shrub_cd_tew   
451   1981    Grassland/Pasture    tew_7  crop3_cd_tew   
452   1984       Woody Wetlands    tew_7   wetf_cd_tew   
453   1985  Herbaceous Wetlands    tew_7  wetem_cd_tew   

                     Index_Loc  
449         Mixed Forest:tew_7  
450            Shrubland:tew_7  
451    Grassland/Pasture:tew_7  
452       Woody Wetlands:tew_7  
453  Herbaceous Wetlands:tew_7  


In [41]:
# Get the ParVal list
t10 = pval2[~pval2.applymap(np.isreal).all(1)]
t11 = t10.select_dtypes(include=['float64','object'])
#print (test2)
t12 = pd.melt(t11, id_vars='description', value_name='ParVal').dropna()
t12 = t12.applymap(lambda x: str(x).strip() if len(str(x).strip()) else None)  # Important - stripped off leading spaces

# Create index location for the values:
t12['Index_Loc'] = t12['description'] + ":" + t12['variable']
t13 = t12[['Index_Loc','ParVal']].copy()
t13.ParVal = t13.ParVal.astype(np.float64)
print (t13.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 454 entries, 0 to 4144
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Index_Loc  454 non-null    object 
 1   ParVal     454 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.6+ KB
None


In [42]:
# Join these together:
parvals_irr = pd.merge(irr5, t13, how='outer', on='Index_Loc').drop_duplicates(['ParName','ParVal'])
print (parvals_irr.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 453
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        217 non-null    int64  
 1   description  217 non-null    object 
 2   variable     217 non-null    object 
 3   ParName      217 non-null    object 
 4   Index_Loc    217 non-null    object 
 5   ParVal       217 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 11.9+ KB
None


In [43]:
# Look for duplicated ParName with different ParVals - fix in spreadsheet:

# ADD CODE HERE to find items, fix in spreadsheet until they are gone.

In [44]:
mask = parvals_irr.ParName.duplicated(keep=False)
dupes = (parvals_irr[mask]).sort_values(['ParName'])
mask2 = dupes.duplicated(['ParName','ParVal'], keep=False)
# Want this to be empty:
realdupes = (dupes[~mask2])
realdupes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        0 non-null      int64  
 1   description  0 non-null      object 
 2   variable     0 non-null      object 
 3   ParName      0 non-null      object 
 4   Index_Loc    0 non-null      object 
 5   ParVal       0 non-null      float64
dtypes: float64(1), int64(1), object(4)
memory usage: 0.0+ bytes


In [45]:
realdupes.to_csv('irrigation_pars_to_clean_up.csv')

## Join LU and IRR pars together:

In [53]:
pars_all = parvals.append(parvals_irr, ignore_index=False).reset_index()
print (pars_all.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   level_0      494 non-null    int64  
 1   index        494 non-null    int64  
 2   description  494 non-null    object 
 3   variable     494 non-null    object 
 4   ParName      494 non-null    object 
 5   Index_Loc    494 non-null    object 
 6   ParVal       494 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 27.1+ KB
None


In [54]:
# Find any duplicates:
#mask = df.B.duplicated(keep=False)
mask = pars_all.ParName.duplicated(keep=False)
print (pars_all[mask])

Empty DataFrame
Columns: [level_0, index, description, variable, ParName, Index_Loc, ParVal]
Index: []


In [55]:
# If needed, remove duplicates
pars_all = pars_all.drop_duplicates(['ParName']).copy()
pars_all['ParName'] = pars_all['ParName'].str.lower()
pars_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494 entries, 0 to 493
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   level_0      494 non-null    int64  
 1   index        494 non-null    int64  
 2   description  494 non-null    object 
 3   variable     494 non-null    object 
 4   ParName      494 non-null    object 
 5   Index_Loc    494 non-null    object 
 6   ParVal       494 non-null    float64
dtypes: float64(1), int64(2), object(4)
memory usage: 30.9+ KB


In [56]:
print (pars_all.tail(10))

     level_0  index                 description variable       ParName  \
484      436   1932       Other Hay/Non Alfalfa    tew_7  crop3_cd_tew   
485      438   1939                   Sugarcane    tew_7  sugcn_cd_tew   
486      442   1970        Developed/Open Space    tew_7  dvlow_cd_tew   
487      444   1972  Developed/Medium Intensity    tew_7  dvmhi_cd_tew   
488      446   1974                      Barren    tew_7   bare_cd_tew   
489      447   1975            Deciduous Forest    tew_7  decid_cd_tew   
490      448   1976            Evergreen Forest    tew_7   evgr_cd_tew   
491      450   1979                   Shrubland    tew_7  shrub_cd_tew   
492      452   1984              Woody Wetlands    tew_7   wetf_cd_tew   
493      453   1985         Herbaceous Wetlands    tew_7  wetem_cd_tew   

                            Index_Loc  ParVal  
484       Other Hay/Non Alfalfa:tew_7   1.063  
485                   Sugarcane:tew_7   1.063  
486        Developed/Open Space:tew_7   0

In [57]:
print (pars_all.ParName.unique())

['bare_a_cn' 'bare_b_cn' 'bare_c_cn' 'bare_d_cn' 'bare_a_mni' 'bare_b_mni'
 'bare_c_mni' 'bare_d_mni' 'bare_ad_mni' 'bare_bd_mni' 'bare_cd_mni'
 'bare_a_rz' 'bare_b_rz' 'bare_c_rz' 'bare_d_rz' 'bare_ad_rz' 'bare_bd_rz'
 'bare_cd_rz' 'crop1_a_cn' 'crop1_b_cn' 'crop1_c_cn' 'crop1_d_cn'
 'crop1_a_mni' 'crop1_b_mni' 'crop1_c_mni' 'crop1_d_mni' 'crop1_ad_mni'
 'crop1_bd_mni' 'crop1_cd_mni' 'crop1_a_rz' 'crop1_b_rz' 'crop1_c_rz'
 'crop1_d_rz' 'crop1_ad_rz' 'crop1_bd_rz' 'crop1_cd_rz' 'cot_a_cn'
 'cot_b_cn' 'cot_c_cn' 'cot_d_cn' 'cot_a_mni' 'cot_b_mni' 'cot_c_mni'
 'cot_d_mni' 'cot_ad_mni' 'cot_bd_mni' 'cot_cd_mni' 'cot_a_rz' 'cot_b_rz'
 'cot_c_rz' 'cot_d_rz' 'cot_ad_rz' 'cot_bd_rz' 'cot_cd_rz' 'crop2_a_cn'
 'crop2_b_cn' 'crop2_c_cn' 'crop2_d_cn' 'crop2_a_mni' 'crop2_b_mni'
 'crop2_c_mni' 'crop2_d_mni' 'crop2_ad_mni' 'crop2_bd_mni' 'crop2_cd_mni'
 'crop2_a_rz' 'crop2_b_rz' 'crop2_c_rz' 'crop2_d_rz' 'crop2_ad_rz'
 'crop2_bd_rz' 'crop2_cd_rz' 'decid_a_cn' 'decid_b_cn' 'decid_c_cn'
 'decid_d_cn'

### Need to set parameter upper and lower bounds, par groups

### Par Groups:

In [58]:
# First, the PAR GROUPS:
pars_all['ParGroup'] = pars_all['ParName'].str.split("_").str[-1]
print (pars_all.tail())

     level_0  index          description variable       ParName  \
489      447   1975     Deciduous Forest    tew_7  decid_cd_tew   
490      448   1976     Evergreen Forest    tew_7   evgr_cd_tew   
491      450   1979            Shrubland    tew_7  shrub_cd_tew   
492      452   1984       Woody Wetlands    tew_7   wetf_cd_tew   
493      453   1985  Herbaceous Wetlands    tew_7  wetem_cd_tew   

                     Index_Loc  ParVal ParGroup  
489     Deciduous Forest:tew_7   0.410      tew  
490     Evergreen Forest:tew_7   0.410      tew  
491            Shrubland:tew_7   0.410      tew  
492       Woody Wetlands:tew_7   0.620      tew  
493  Herbaceous Wetlands:tew_7   1.063      tew  


In [59]:
print (pars_all['ParGroup'].unique())
print (pars_all.info())

['cn' 'mni' 'rz' 'height' 'kcb-mid' 'rew' 'tew']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 494 entries, 0 to 493
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   level_0      494 non-null    int64  
 1   index        494 non-null    int64  
 2   description  494 non-null    object 
 3   variable     494 non-null    object 
 4   ParName      494 non-null    object 
 5   Index_Loc    494 non-null    object 
 6   ParVal       494 non-null    float64
 7   ParGroup     494 non-null    object 
dtypes: float64(1), int64(2), object(5)
memory usage: 34.7+ KB
None


### Upper and Lower Bounds:

In [60]:
# Set Upper and Lower bounds for Curve Numbers:
# Arbitrary range of 30 - 99 in total. 
pars_all['ParUBND'] = -99
pars_all['ParLBND'] = -999
maxCN = 99.0
minCN = 30.0
# Upper BND =  (ParVal + (MaxCN - ParVal)/2)
pars_all['tmp'] = maxCN - pars_all['ParVal']
pars_all['tmp'] = pars_all['tmp'] / 2
pars_all['tmp'] = pars_all.ParVal + pars_all.tmp
pars_all.loc[pars_all.ParGroup == 'cn', 'ParUBND'] = pars_all['tmp']
# Lower BND =  (ParVal - (ParVal - MinCN)/2)
pars_all['tmp'] = pars_all['ParVal'] - minCN
pars_all['tmp'] = pars_all['tmp'] / 2
pars_all['tmp'] = pars_all.ParVal - pars_all.tmp
pars_all.loc[pars_all.ParGroup == 'cn', 'ParLBND'] = pars_all['tmp']

print (pars_all.head())

   level_0  index description         variable     ParName  \
0        0     69      Barren             cn_1   bare_a_cn   
1        1    196      Barren             cn_2   bare_b_cn   
2        2    323      Barren             cn_3   bare_c_cn   
3        3    450      Barren             cn_4   bare_d_cn   
4        7    958      Barren  max_net_infil_1  bare_a_mni   

                Index_Loc   ParVal ParGroup  ParUBND   ParLBND      tmp  
0             Barren:cn_1  81.4000       cn  90.2000   55.7000  55.7000  
1             Barren:cn_2  88.4308       cn  93.7154   59.2154  59.2154  
2             Barren:cn_3  92.3554       cn  95.6777   61.1777  61.1777  
3             Barren:cn_4  93.8992       cn  96.4496   61.9496  61.9496  
4  Barren:max_net_infil_1   3.7500      mni -99.0000 -999.0000  16.8750  


In [61]:
# Set upper and lower bounds for ROOTING ZONE DEPTHS
# Just use a multiplier, nothing fancy:
pars_all.loc[pars_all.ParGroup == 'rz', 'ParUBND'] = pars_all['ParVal'] * 1.5
pars_all.loc[pars_all.ParGroup == 'rz', 'ParLBND'] = pars_all['ParVal'] * 0.5

print (pars_all.head())

   level_0  index description         variable     ParName  \
0        0     69      Barren             cn_1   bare_a_cn   
1        1    196      Barren             cn_2   bare_b_cn   
2        2    323      Barren             cn_3   bare_c_cn   
3        3    450      Barren             cn_4   bare_d_cn   
4        7    958      Barren  max_net_infil_1  bare_a_mni   

                Index_Loc   ParVal ParGroup  ParUBND   ParLBND      tmp  
0             Barren:cn_1  81.4000       cn  90.2000   55.7000  55.7000  
1             Barren:cn_2  88.4308       cn  93.7154   59.2154  59.2154  
2             Barren:cn_3  92.3554       cn  95.6777   61.1777  61.1777  
3             Barren:cn_4  93.8992       cn  96.4496   61.9496  61.9496  
4  Barren:max_net_infil_1   3.7500      mni -99.0000 -999.0000  16.8750  


In [62]:
# Set for PLANT HEIGHTS:
# Use same thing: simple multipliers
pars_all.loc[pars_all.ParGroup == 'height', 'ParUBND'] = pars_all['ParVal'] * 1.5
pars_all.loc[pars_all.ParGroup == 'height', 'ParLBND'] = pars_all['ParVal'] * 0.5

# Set for Kcb-Mid values:
# Use same thing: simple multipliers
pars_all.loc[pars_all.ParGroup == 'kcb-mid', 'ParUBND'] = pars_all['ParVal'] * 1.25
pars_all.loc[pars_all.ParGroup == 'kcb-mid', 'ParLBND'] = pars_all['ParVal'] * 0.75

# Set for REW and TEW settings:
pars_all.loc[pars_all.ParGroup == 'rew', 'ParUBND'] = pars_all['ParVal'] * 1.5
pars_all.loc[pars_all.ParGroup == 'rew', 'ParLBND'] = pars_all['ParVal'] * 0.1
pars_all.loc[pars_all.ParGroup == 'tew', 'ParUBND'] = pars_all['ParVal'] * 1.5
pars_all.loc[pars_all.ParGroup == 'tew', 'ParLBND'] = pars_all['ParVal'] * 0.1

print (pars_all.tail())

     level_0  index          description variable       ParName  \
489      447   1975     Deciduous Forest    tew_7  decid_cd_tew   
490      448   1976     Evergreen Forest    tew_7   evgr_cd_tew   
491      450   1979            Shrubland    tew_7  shrub_cd_tew   
492      452   1984       Woody Wetlands    tew_7   wetf_cd_tew   
493      453   1985  Herbaceous Wetlands    tew_7  wetem_cd_tew   

                     Index_Loc  ParVal ParGroup  ParUBND  ParLBND      tmp  
489     Deciduous Forest:tew_7   0.410      tew   0.6150   0.0410  15.2050  
490     Evergreen Forest:tew_7   0.410      tew   0.6150   0.0410  15.2050  
491            Shrubland:tew_7   0.410      tew   0.6150   0.0410  15.2050  
492       Woody Wetlands:tew_7   0.620      tew   0.9300   0.0620  15.3100  
493  Herbaceous Wetlands:tew_7   1.063      tew   1.5945   0.1063  15.5315  


In [ ]:
# Max Net Infiltration: same thing, simple multipliers:
pars_all.loc[pars_all.ParGroup == 'mni', 'ParUBND'] = pars_all['ParVal'] * 2
pars_all.loc[pars_all.ParGroup == 'mni', 'ParLBND'] = pars_all['ParVal'] * 0.25

In [64]:
# Find any that haven't been set yet:

pars_all.loc[pars_all['ParUBND'] < 0]

,level_0,index,description,variable,ParName,Index_Loc,ParVal,ParGroup,ParUBND,ParLBND,tmp


### Adjustable:

In [65]:
pars_all['Adjust'] = 'YES'
print (pars_all.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494 entries, 0 to 493
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   level_0      494 non-null    int64  
 1   index        494 non-null    int64  
 2   description  494 non-null    object 
 3   variable     494 non-null    object 
 4   ParName      494 non-null    object 
 5   Index_Loc    494 non-null    object 
 6   ParVal       494 non-null    float64
 7   ParGroup     494 non-null    object 
 8   ParUBND      494 non-null    float64
 9   ParLBND      494 non-null    float64
 10  tmp          494 non-null    float64
 11  Adjust       494 non-null    object 
dtypes: float64(4), int64(2), object(6)
memory usage: 70.2+ KB
None


### Clean up for exporting:

In [66]:
pars_exp = pars_all[['ParName','ParGroup','ParVal','ParUBND','ParLBND','Adjust']].copy()

In [67]:
outdir = r"E:\UMID_Data\0188_offline\modeldev\PARS"
pars_exp.to_csv(os.path.join(outdir,'MAP_parlist_MOREPARS.csv'))